In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model,Sequential
from keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import matplotlib.pyplot as plt
#%matplotlib inline


In [2]:
folder_path = "../input/nupurdb2204/DB main/"
train_path = folder_path+"Train/"
valid_path = folder_path+"Validate/"
test_path = folder_path+"Test/"


In [3]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=train_path, target_size=(224,224), batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=valid_path, target_size=(224,224), batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=test_path, target_size=(224,224), batch_size=10)


/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:179: UserWarning: Using ".tiff" files with multiple bands will cause distortion. Please verify your output.
  warnings.warn('Using ".tiff" files with multiple bands '


Found 7000 images belonging to 7 classes.
Found 1260 images belonging to 7 classes.
Found 2100 images belonging to 7 classes.


In [4]:
mobile = tf.keras.applications.mobilenet.MobileNet()
mobile.summary()

2022-05-25 14:27:30.691929: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-25 14:27:30.834935: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-25 14:27:30.836142: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-25 14:27:30.838003: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

17235968/17225924 [==============================] - 1s 0us/step
Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________

In [5]:
x = mobile.layers[-6].output
output = Dense(units=7, activation='softmax')(x)


In [6]:
model = Sequential()
model = Model(inputs=mobile.input, outputs=output)


In [7]:
for layer in model.layers[:-23]:
    layer.trainable = False 
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0     

In [8]:

model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy', metrics=['accuracy'])


In [9]:
#Run for more epochs (-30) to see better results
model.fit(x=train_batches, validation_data=valid_batches, epochs=40, verbose=2)

2022-05-25 14:27:37.946725: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/40


2022-05-25 14:27:41.183993: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


700/700 - 45s - loss: 1.4571 - accuracy: 0.4593 - val_loss: 0.9479 - val_accuracy: 0.6706
Epoch 2/40
700/700 - 19s - loss: 0.8893 - accuracy: 0.6910 - val_loss: 0.7448 - val_accuracy: 0.7579
Epoch 3/40
700/700 - 20s - loss: 0.6252 - accuracy: 0.7971 - val_loss: 0.6185 - val_accuracy: 0.7825
Epoch 4/40
700/700 - 20s - loss: 0.4374 - accuracy: 0.8736 - val_loss: 0.8030 - val_accuracy: 0.6968
Epoch 5/40
700/700 - 20s - loss: 0.3336 - accuracy: 0.9064 - val_loss: 0.4850 - val_accuracy: 0.8206
Epoch 6/40
700/700 - 20s - loss: 0.2604 - accuracy: 0.9341 - val_loss: 0.3263 - val_accuracy: 0.9333
Epoch 7/40
700/700 - 21s - loss: 0.2082 - accuracy: 0.9506 - val_loss: 0.2558 - val_accuracy: 0.9516
Epoch 8/40
700/700 - 20s - loss: 0.1770 - accuracy: 0.9550 - val_loss: 0.2832 - val_accuracy: 0.9397
Epoch 9/40
700/700 - 22s - loss: 0.1593 - accuracy: 0.9567 - val_loss: 0.2974 - val_accuracy: 0.9437
Epoch 10/40
700/700 - 19s - loss: 0.1471 - accuracy: 0.9624 - val_loss: 0.3579 - val_accuracy: 0.9405


In [10]:
model.save("./mobilenet_model.h5")